# 🍏 Fitness Fun: Azure Functions + AI Agent Tutorial (.NET) 🍎

This notebook shows how to use **Azure Functions** with the **Azure AI Foundry** Agents SDK for **.NET** (`Azure.AI.Agents.Persistent`, `Azure.Identity`). We'll:

1. **Set up** an Azure Function that listens on a storage queue.
2. **Create** an AI Agent that uses this function as a tool.
3. **Send** a prompt to the agent; it calls the function.
4. **Retrieve** the processed result from the output queue.

All with a fun, health-and-fitness-themed example! We'll keep it whimsical, but remember:

### ⚠️ Important Disclaimer
> **This example is for demonstration purposes only and does not provide genuine medical or health advice.** Always consult a professional for real medical or fitness advice.

## Prerequisites
1. Azure Subscription & **Azure AI Foundry** project (`PROJECT_ENDPOINT` or `AI_FOUNDRY_PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`).
2. **Azure Functions** environment or local emulator (Azurite) + Storage Queue.
3. **.NET 8+** with **dotnet-interactive** notebook support.
4. NuGet packages:
   - `Azure.AI.Agents.Persistent` (preview)
   - `Azure.Identity`
   - Optional: `DotNetEnv` for `.env` loading

## Overview
1. **Azure Function** is set up to read messages from an **input queue** and write responses to an **output queue**.
2. **AI Agent** is created with an `AzureFunctionTool` that references these queues.
3. **User** provides a question or command; the agent decides whether or not to call the function.
4. The agent sends a message to the **input queue**, which triggers the function.
5. **Azure Function** processes the message, sends back a response to the **output queue**.
6. The agent picks up the response from the output queue.
7. The **User** sees the final answer from the agent.

<img src="./seq-diagrams/6-az-function.png" width="75%"/>

## 1. Azure Function Setup (C# Example)
Below is an **isolated worker** Azure Functions sample that reads from `azure-function-foo-input` and writes to `azure-function-tool-output`.

```csharp
using System;
using System.Text.Json;
using Azure.Identity;
using System.Threading.Tasks;
using Azure.Storage.Queues;
using Azure.Storage.Queues.Models;
using Microsoft.Azure.Functions.Worker;
using Microsoft.Extensions.Logging;

namespace azure_function;

public class FooReply
{
    private readonly QueueClient _outputQueue;
    private readonly ILogger<FooReply> _logger;

    public FooReply(
        ILogger<FooReply> logger)
    {
        _logger = logger;

        var storageServiceEndpoint = Environment.GetEnvironmentVariable("STORAGE_SERVICE_ENDPOINT")
            ?? throw new InvalidOperationException("Missing STORAGE_SERVICE_ENDPOINT");

        _outputQueue = new QueueClient(
            new Uri($"{storageServiceEndpoint}/azure-function-tool-output"),
            new DefaultAzureCredential(),
            new QueueClientOptions { MessageEncoding = QueueMessageEncoding.Base64 });

        _outputQueue.CreateIfNotExists();
    }

    [Function(nameof(FooReply))]
    public async Task Run([QueueTrigger("azure-function-foo-input", Connection = "STORAGE_SERVICE_ENDPOINT")] QueueMessage message)
    {
        _logger.LogInformation("Azure Function triggered with a queue item.");

        using JsonDocument doc = JsonDocument.Parse(message.MessageText);
        var root = doc.RootElement;
        var userQuery = root.TryGetProperty("query", out var q) ? q.GetString() ?? string.Empty : string.Empty;
        var correlationId = root.TryGetProperty("CorrelationId", out var c) ? c.GetString() ?? string.Empty : string.Empty;

        var result = new
        {
            FooReply = $"This is Foo, responding to: {userQuery}! Stay strong 💪!",
            CorrelationId = correlationId
        };

        await _outputQueue.SendMessageAsync(JsonSerializer.Serialize(result));
        _logger.LogInformation("Sent message: {Result}", JsonSerializer.Serialize(result));
    }
}
```

**Notes**
- The input queue name is `azure-function-foo-input`.
- The output queue name is `azure-function-tool-output`.
- `STORAGE_SERVICE_ENDPOINT` should be an endpoint like `https://<account>.queue.core.windows.net`.
- If using connection strings, set `Connection = "AzureWebJobsStorage"` and instantiate `QueueClient` with the connection string.

## 2. Notebook Setup (.NET)

Now let's continue here in our notebook environment. We'll:
1. Import the required nuget packages.
2. Initialize `PersistentAgentsClient`
3. Create the Azure Function tool definition and the Agent

In [ ]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.8"
#r "nuget: Azure.Identity"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using System.Text.Json;
using dotenv.net;
using System.IO;

// Load environment variables
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
PersistentAgentsClient client;
try
{
    var credentialOptions = new DefaultAzureCredentialOptions
    {
        ExcludeManagedIdentityCredential = true,
        ExcludeEnvironmentCredential = true
    };
    client = new PersistentAgentsClient(projectEndpoint, new DefaultAzureCredential(credentialOptions));
    Console.WriteLine("✅ Successfully initialized PersistentAgentsClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Failed to initialize PersistentAgentsClient: {ex.Message}");
}

### Create Agent with Azure Function Tool
We'll define a tool that references our function name (`foo` or `FooReply` from the sample) and the input + output queues. In this example we'll store the queue endpoint in an env variable called `STORAGE_SERVICE_ENDPOINT`.

You can addapt it to your own naming scheme. The agent instructions tell it to use the function whenever it sees certain keywords, or you could just let it call the function on its own.

In [ ]:
// create the QueueStorageServiceEndpoint value
string storageServiceEndpoint = Environment.GetEnvironmentVariable("STORAGE_SERVICE_ENDPOINT");

if (string.IsNullOrEmpty(storageServiceEndpoint))
{
    Console.WriteLine("❌ STORAGE_SERVICE_ENDPOINT environment variable is not set.");
    throw new InvalidOperationException("STORAGE_SERVICE_ENDPOINT environment variable is not set.");
}

AzureFunctionToolDefinition azureFunctionTool = new(
    name: "foo",
    description: "Get comedic or silly advice from 'Foo'.",
    inputBinding: new AzureFunctionBinding(
        new AzureFunctionStorageQueue(
            queueName: "azure-function-foo-input",
            storageServiceEndpoint: storageServiceEndpoint)
    ),
    outputBinding: new AzureFunctionBinding(
        new AzureFunctionStorageQueue(
            queueName: "azure-function-tool-output",
            storageServiceEndpoint: storageServiceEndpoint)
    ),
    parameters: BinaryData.FromObjectAsJson(
        new
        {
            type = "object",
            properties = new
            {
                query = new { type = "string", description = "The question to ask Foo." },
                outputqueueuri = new { type = "string", description = "The output queue URI." }
            }
        },
        new JsonSerializerOptions(JsonSerializerDefaults.Web)
    )
);

PersistentAgent? agent = client.Administration.CreateAgent(
    model: modelDeployment,
    name: "azure-function-agent-foo",
    instructions:
        "You are a helpful health and fitness support agent.\n" +
        "If the user says 'What would foo say?' then call the foo function.\n" +
        $"Always specify the outputqueueuri as '{storageServiceEndpoint}/azure-function-tool-output'.\n" +
        "Respond with 'Foo says: <response>' after the tool call.",
    tools: [ azureFunctionTool ]
);

Console.WriteLine($"🎉 Created agent, agent ID: {agent?.Id}");

## 3. Test the Agent

Now let's simulate a user message that triggers the function call. We'll create a conversation **thread**, post a user question that includes "What would foo say?", then run the agent.

The Agent Service will place a message on the `azure-function-foo-input` queue. The function will handle it and place a response in `azure-function-tool-output`. The agent will pick that up automatically and produce a final answer.

In [ ]:
using Azure;
using Azure.AI.Agents.Persistent;
using System.Threading.Tasks;

static async Task<(PersistentAgentThread thread, ThreadRun run)> RunFooQuestion(
    string userQuestion,
    string agentId,
    PersistentAgentsClient client)
{
    // 1. Create a new thread
    var thread = await client.Threads.CreateThreadAsync();
    Console.WriteLine($"📝 Created thread, thread ID: {thread.Id}");

    // 2. Add a user message to the thread
    var message = await client.Messages.CreateMessageAsync(
        threadId: thread.Id,
        role: MessageRole.User,
        content: userQuestion
    );
    Console.WriteLine($"💬 Created user message, ID: {message.Id}");

    // 3. Create a run for the thread
    var run = await client.Runs.CreateRunAsync(thread.Id, agentId);
    Console.WriteLine($"🤖 Run created, status: {run.Status}");

    // 4. Poll for run completion
    do
    {
        await Task.Delay(TimeSpan.FromSeconds(1));
        run = await client.Runs.GetRunAsync(thread.Id, run.Id);
        Console.WriteLine($"🔄 Run status: {run.Status}");
    }
    while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress);

    Console.WriteLine($"🤖 Run completed with status: {run.Status}");

    if (run.Status == RunStatus.Failed)
    {
        Console.WriteLine($"Run failed: {run.LastError?.Message}");
    }

    // 5. Retrieve and display the messages in the thread
    Console.WriteLine("\n🗣️ Conversation:");
    await foreach (PersistentThreadMessage m in client.Messages.GetMessagesAsync(
        threadId: thread.Id,
        order: ListSortOrder.Ascending))
    {
        Console.Write($"{m.Role.ToString().ToUpper()}: ");
        foreach (MessageContent content in m.ContentItems)
        {
            if (content is MessageTextContent text)
            {
                Console.WriteLine(text.Text);
            }
        }
    }

    return (thread, run);
}

if (agent is not null)
{
    var (myThread, myRun) = await RunFooQuestion(
        userQuestion: "What is the best post-workout snack? What would foo say?",
        agentId: agent.Id,
        client: client
    );
}

## 4. Cleanup
We'll remove the agent when done. In real scenarios, you might keep your agent for repeated usage.

In [ ]:
if (agent is not null)
{
    try
    {
        await client.Administration.DeleteAgentAsync(agent.Id);
        Console.WriteLine($"🗑️ Deleted agent: {agent.Name}");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error deleting agent: {ex.Message}");
    }
}

# 🎉 Congratulations!
You just saw how to combine **Azure Functions** with **AI Agent Service** to create a dynamic, queue-driven workflow. In this whimsical example, your function returned comedic "Foo says..." lines, but in real applications, you can harness the power of Azure Functions to run anything from **database lookups** to **complex calculations**, returning the result seamlessly to your AI agent.

## Next Steps
- **Add OpenTelemetry** to gather end-to-end tracing across your function and agent.
- Incorporate an **evaluation** pipeline with `Microsoft.Extensions.AI.Evaluation` SDK to measure how well your agent + function workflow addresses user queries.
- Explore **parallel function calls** or more advanced logic in your Azure Functions.

Happy coding and stay fit! 🤸